In [ ]:
# import dependencies
# import API key from file

In [ ]:
# generate a list 500 random coordinate pairs that are well distributed across the coordinate range
# import citipy, create a list of cities from the random coordinate pairs

In [ ]:
#set up loop for each city in citylist
    # make an api call to openweathermap's API
    # return lat, long, temperatureF cloudiness%, Humidity%, windspeedmph
    #apendd data for each city to an existing list
    #print city name and number to a log file

#save data to a csv file

In [ ]:

#for each variable in list
    #plot scatter plot vs latitude
    #export each plot to a png file
    #if lat >o
        # plot all possible scatter plots on new plots
        #run linear regression for each plot
        #add regression results to the plot
        #export plot to png file
    #else
        #do the same as above on separate plots
        